# Hybrid Streaming Optimizer API Demo


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/06_streaming/05_hybrid_streaming_api.ipynb)

This notebook demonstrates how to use `method='hybrid_streaming'` with both
`curve_fit()` and `curve_fit_large()` functions.

The hybrid streaming optimizer provides:
- Parameter normalization for better gradient signals
- L-BFGS warmup for robust initial convergence
- **4-layer defense strategy** for warmup divergence prevention (v0.3.6+)
- Streaming Gauss-Newton for exact covariance computation
- Automatic memory management for large datasets

In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import jax.numpy as jnp
import numpy as np

from nlsq import (
    HybridStreamingConfig,
    curve_fit,
    curve_fit_large,
    get_defense_telemetry,
    reset_defense_telemetry,
)

In [ ]:
def exponential_decay(x, a, b, c):
    """Three-parameter exponential decay model."""
    return a * jnp.exp(-b * x) + c

In [ ]:
# Generate synthetic data
np.random.seed(42)
x = np.linspace(0, 10, 2000)
true_params = np.array([5.0, 0.5, 1.0])
y_true = exponential_decay(x, *true_params)
y = y_true + np.random.normal(0, 0.1, len(x))

print(f"Dataset: {len(x)} samples")
print(f"True parameters: a={true_params[0]}, b={true_params[1]}, c={true_params[2]}")

## Example 1: Basic usage with curve_fit()

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

# Unpack result
popt, pcov = result

print(f"\nFitted parameters: {popt}")
print(f"True parameters:   {true_params}")
print(f"Parameter errors:  {np.abs(popt - true_params)}")
print(f"\nCovariance matrix diagonal: {np.diag(pcov)}")
print(f"Parameter std errors: {np.sqrt(np.diag(pcov))}")

## Example 2: With parameter bounds

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    bounds=([0, 0, 0], [10, 2, 5]),
    method='hybrid_streaming',
    verbose=0,  # Silent mode
)

popt, pcov = result
print(f"Fitted parameters (bounded): {popt}")
print(f"Within bounds: {np.all(popt >= [0, 0, 0]) and np.all(popt <= [10, 2, 5])}")

## Example 3: Large dataset with curve_fit_large()

In [ ]:
# Generate larger dataset
x_large = np.linspace(0, 10, 10000)
y_large = exponential_decay(x_large, *true_params) + np.random.normal(0, 0.1, len(x_large))

popt, pcov = curve_fit_large(
    exponential_decay,
    x_large,
    y_large,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

print(f"\nFitted parameters (large dataset): {popt}")
print(f"True parameters:                   {true_params}")
print(f"Parameter errors:                  {np.abs(popt - true_params)}")

## Example 4: Custom configuration via kwargs

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
    # HybridStreamingConfig overrides:
    warmup_iterations=300,
    normalization_strategy='p0',  # 'bounds' requires explicit bounds
    phase2_max_iterations=100,
)

popt, pcov = result
print(f"Fitted parameters (custom config): {popt}")
print(f"Result attributes available: {list(result.keys())[:10]}")

## Example 5: Accessing full result details

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
)

print(f"Success: {result.success}")
print(f"Message: {result.message}")
print(f"Final cost: {getattr(result, 'cost', 'N/A')}")

if hasattr(result, 'streaming_diagnostics') and result.streaming_diagnostics:
    diag = result.streaming_diagnostics
    print("\nStreaming diagnostics available:")
    print(f"  Keys: {list(diag.keys())}")

---

## Example 6: 4-Layer Defense Strategy (v0.3.6+)

The hybrid streaming optimizer includes a **4-layer defense strategy** that prevents L-BFGS warmup divergence when initial parameters are already near optimal.

### The 4 Defense Layers:
1. **Layer 1: Warm Start Detection** - Skips warmup if initial loss < 1% of data variance
2. **Layer 2: Adaptive Step Size** - Scales step size based on initial fit quality
3. **Layer 3: Cost-Increase Guard** - Aborts if loss increases > 5% from initial
4. **Layer 4: Step Clipping** - Limits parameter update magnitude (max norm 0.1)

In [ ]:
# Demonstrate defense layers with near-optimal starting point
reset_defense_telemetry()

# Near-optimal initial guess (simulating refinement scenario)
near_optimal_p0 = true_params * np.array([1.01, 0.99, 1.005])
print(f"Near-optimal p0: {near_optimal_p0}")
print(f"True params:     {true_params}")

# Defense layers are enabled by default
popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=near_optimal_p0,
    method='hybrid_streaming',
    verbose=0,
)

# Check telemetry
telemetry = get_defense_telemetry()
summary = telemetry.get_summary()
rates = telemetry.get_trigger_rates()

print(f"\nFitted params: {popt}")
print("\n--- Defense Layer Telemetry ---")
print(f"Layer 1 (warm start) rate: {rates.get('layer1_warm_start_rate', 0):.1f}%")
print(f"Layer 2 LR modes: {summary.get("layer2_lr_mode_counts", {})}")
print(f"Layer 3 (cost guard) triggers: {summary.get("layer3_cost_guard_triggers", 0)}")
print(f"Layer 4 (step clip) triggers: {summary.get("layer4_clip_triggers", 0)}")

### Example 7: Using Defense Layer Presets

In [ ]:
# Available preset configurations
presets = {
    "Default": HybridStreamingConfig(),
    "Strict (refinement)": HybridStreamingConfig.defense_strict(),
    "Relaxed (exploration)": HybridStreamingConfig.defense_relaxed(),
    "Disabled (pre-0.3.6)": HybridStreamingConfig.defense_disabled(),
    "Scientific": HybridStreamingConfig.scientific_default(),
}

print("Defense Layer Presets:")
print(f"{'Preset':<25} {'L1':<5} {'L2':<5} {'L3':<5} {'L4':<5}")
print("-" * 50)

for name, config in presets.items():
    print(f"{name:<25} {'ON' if config.enable_warm_start_detection else 'OFF':<5} "
          f"{'ON' if config.enable_adaptive_warmup_lr else 'OFF':<5} "
          f"{'ON' if config.enable_cost_guard else 'OFF':<5} "
          f"{'ON' if config.enable_step_clipping else 'OFF':<5}")

In [ ]:
# Use a preset for warm start refinement
config = HybridStreamingConfig.defense_strict()

popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=near_optimal_p0,
    method='hybrid_streaming',
    config=config,  # Pass preset config
    verbose=0,
)

print(f"Fitted with defense_strict(): {popt}")

### Example 8: Custom Defense Configuration

In [ ]:
# Fine-tune defense layers for specific needs
custom_config = HybridStreamingConfig(
    # Layer 1: Stricter warm start threshold
    enable_warm_start_detection=True,
    warm_start_threshold=0.005,  # 0.5% instead of 1%

    # Layer 2: Conservative learning rates
    enable_adaptive_warmup_lr=True,
    warmup_lr_refinement=1e-7,
    warmup_lr_careful=1e-6,

    # Layer 3: Tighter cost tolerance
    enable_cost_guard=True,
    cost_increase_tolerance=0.02,  # 2% instead of 5%

    # Layer 4: Smaller step limit
    enable_step_clipping=True,
    max_warmup_step_size=0.05,
)

popt, pcov = curve_fit(
    exponential_decay,
    x, y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    config=custom_config,
    verbose=0,
)

print(f"Fitted with custom defense config: {popt}")

### Example 9: Production Monitoring with Telemetry

In [ ]:
# Production monitoring example
reset_defense_telemetry()

# Simulate batch of fits with varying quality
for i in range(3):
    noise = 0.01 if i < 3 else (0.3 if i < 7 else 1.0)
    p0 = true_params * (1 + np.random.uniform(-noise, noise, 3))

    curve_fit(
        exponential_decay, x, y, p0=p0,
        method='hybrid_streaming',
        verbose=0,
    )

# Get monitoring report
telemetry = get_defense_telemetry()
rates = telemetry.get_trigger_rates()

print("--- Production Monitoring Report (10 fits) ---")
print(f"Layer 1 (warm start):     {rates.get('layer1_warm_start_rate', 0):.1f}%")
print(f"Layer 2 (refinement LR):  {rates.get('layer2_refinement_rate', 0):.1f}%")
print(f"Layer 2 (careful LR):     {rates.get('layer2_careful_rate', 0):.1f}%")
print(f"Layer 2 (exploration LR): {rates.get('layer2_exploration_rate', 0):.1f}%")
print(f"Layer 3 (cost guard):     {rates.get('layer3_cost_guard_rate', 0):.1f}%")
print(f"Layer 4 (step clipping):  {rates.get('layer4_clip_rate', 0):.1f}%")

# Export Prometheus-compatible metrics
print("\n--- Prometheus Metrics ---")
for name, value in telemetry.export_metrics().items():
    print(f"{name}: {value}")

---

## Summary

The hybrid streaming optimizer combines:
- **Parameter normalization** for better gradient signals when parameters have different scales
- **L-BFGS warmup** for robust initial convergence from poor initial guesses
- **4-layer defense strategy** (v0.3.6+) preventing warmup divergence near optimal
- **Streaming Gauss-Newton** for fast second-order convergence near the optimum
- **Exact covariance computation** for reliable uncertainty estimates

### Use `method='hybrid_streaming'` when:
- Parameters span many orders of magnitude
- Large datasets (100K+ points) with memory constraints
- Need production-quality uncertainty estimates
- Standard optimizers converge slowly
- Refining parameters from previous fits (warm starts)

### Defense Layer Presets:
| Scenario | Preset |
|----------|--------|
| Default usage | `HybridStreamingConfig()` |
| Warm start refinement | `HybridStreamingConfig.defense_strict()` |
| Exploration | `HybridStreamingConfig.defense_relaxed()` |
| Regression testing | `HybridStreamingConfig.defense_disabled()` |
| Scientific computing | `HybridStreamingConfig.scientific_default()` |

### Next Steps:
- [Defense Layers Demo](../05_feature_demos/defense_layers_demo.ipynb) - Deep dive into defense layers
- [Troubleshooting Guide](../03_advanced/troubleshooting_guide.ipynb) - Common issues and solutions